In [1]:
import time
import math 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re

In [3]:
# browser = Browser('edge')

In [4]:
#url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=20"
#browser.visit(url)

In [5]:
#html = browser.html
#html_soup = soup(html, 'html.parser')

In [6]:
# product_info = html_soup.find_all(class_='nl-product__content')
# product_data1 = []

# for product in product_info:
#     product_name_element = product.find('div', class_='nl-product-card__title')
#     original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
#     clearance_price_element = product.find('span', class_='nl-price--total--red')
#     rating_element = product.find('div', class_='bv_text')
#     image_element = product.find('div', class_='nl-product-card__image-wrap')
#     anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
#     img_tag = product.find('img', attrs={'data-src': True})

#     product_name = product_name_element.text.strip() if product_name_element else 'N/A'
#     original_price = original_price_element.text.strip() if original_price_element else 'N/A'
#     clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
#     rating = rating_element.text.strip() if rating_element else None
#     product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
#     product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
#     product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
#     product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

#     if original_price == 'N/A':
#         alt_original_price_element = product.find('span', class_='nl-price--total')
#         original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

#     clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
#     original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

#     product_data1.append({
#         'Product Name': product_name,
#         'Original Price': original_price,
#         'Clearance Price': clearance_price,
#         'Rating': rating,
#         'Product Code': product_code,
#         'Link': product_link, 
#         'Image Link': product_image_link
#     })

# print(product_data1)

In [7]:
# df1 = pd.DataFrame(product_data1)

# df1.sample(24)

In [8]:
# df1['Percentage Off'] = round(((df1['Original Price'] - df1['Clearance Price']) / df1['Original Price']) * 100)

# df1.sample(24)

In [9]:
# from urllib.parse import urlparse

# def extract_category(link_url):
#     parsed_url = urlparse(link_url)
#     path_components = parsed_url.path.split('/')
#     category = '/'.join(path_components[3:4])
#     return category

# df1['Product Category'] = df1['Image Link'].apply(extract_category)

# df1

In [10]:
base_url = "https://www.canadiantire.ca/en/promotions/clearance.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(base_url)
time.sleep(5)

html_content = driver.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver.page_source
    html_soup = soup(html_content, 'html.parser')

    product_info = html_soup.find_all(class_='nl-product__content')

    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 1157
Total number of pages: 49


In [11]:
from urllib.parse import urlparse

def extract_category(link_url):
    parsed_url = urlparse(link_url)
    path_components = parsed_url.path.split('/')
    category = '/'.join(path_components[3:4])
    return category

In [12]:
clearance_df = pd.DataFrame(product_data)
clearance_df['Percentage Off'] = round(((clearance_df['Original Price'] - clearance_df['Clearance Price']) / clearance_df['Original Price']) * 100)
clearance_df['Product Category'] = clearance_df['Image Link'].apply(extract_category)

In [13]:
base_url = "https://www.canadiantire.ca/en/promotions/hot-sale.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver2 = webdriver.Chrome(service=service)

driver2.get(base_url)
time.sleep(5)
html_content = driver2.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver2.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver2.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info = html_soup.find_all(class_='nl-product__content')

    for product in sale_product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver2.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 1949
Total number of pages: 82


In [14]:
sale_df = pd.DataFrame(sale_product_data)
sale_df['Percentage Off'] = round(((sale_df['Original Price'] - sale_df['Sale Price']) / sale_df['Original Price']) * 100)
sale_df['Product Category'] = sale_df['Image Link'].apply(extract_category)


In [15]:
base_url = "https://www.canadiantire.ca/en/promotions/red-alert-deals.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver3 = webdriver.Chrome(service=service)

driver3.get(base_url)
time.sleep(5)
html_content = driver3.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data_2 = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver3.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver3, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver3.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info2 = html_soup.find_all(class_='nl-product__content')

    for product in sale_product_info2:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data_2.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver3.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 461
Total number of pages: 20


In [16]:
sale_df2 = pd.DataFrame(sale_product_data_2)
sale_df2['Percentage Off'] = round(((sale_df2['Original Price'] - sale_df2['Sale Price']) / sale_df2['Original Price']) * 100)
sale_df2['Product Category'] = sale_df2['Image Link'].apply(extract_category)

In [17]:
print(len(clearance_df))
clearance_df.sample(5)

1157


,Product Name,Original Price,Clearance Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
761,"Allen 2169 Overshooting & Safety Glasses, Clear",17.49,4.93,4.0,175-8710-6,https://www.canadiantire.ca/en/pdp/allen-2169-...,https://media-www.canadiantire.ca/product/play...,72.0,hunting
568,"Petco Unstuffed Squeak Plush Fox Dog Toy, Large",13.99,7.93,4.3,142-3871-0,https://www.canadiantire.ca/en/pdp/petco-unstu...,https://media-www.canadiantire.ca/product/livi...,43.0,pet-care
1146,"Simoniz Tire Shine and Glass Cleaner Kit, 2-pack",NaN,6.93,0.0,399-8761-2,https://www.canadiantire.ca/en/pdp/simoniz-tir...,https://media-www.canadiantire.ca/product/auto...,NaN,car-care-accessories
450,"Hangman Security Wire Hanger, Security Lips, 4...",5.49,1.43,0.0,161-1041-2,https://www.canadiantire.ca/en/pdp/hangman-sec...,https://media-www.canadiantire.ca/product/fixi...,74.0,hardware
676,Honeywell HE240A Whole House Flow-Through Bypa...,205.99,59.93,2.6,064-3122-0,https://www.canadiantire.ca/en/pdp/honeywell-h...,https://media-www.canadiantire.ca/product/fixi...,71.0,home-environment


In [18]:
print(len(sale_df))
sale_df.sample(5)

1949


,Product Name,Original Price,Sale Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
578,Shop-Vac® 5830448 5.5 SVX2 Peak HP Oval Wet/Dr...,159.99,139.99,3.7,054-0331-4,https://www.canadiantire.ca/en/pdp/shop-vac-58...,https://media-www.canadiantire.ca/product/fixi...,13.0,tools
834,Black & Decker 6-Slice Digital Countertop Conv...,199.99,119.99,3.9,043-0147-0,https://www.canadiantire.ca/en/pdp/black-decke...,https://media-www.canadiantire.ca/product/livi...,40.0,kitchen
1886,Simoniz 3-Layer Water Resistant Car Cover with...,144.99,130.49,0.0,041-2261-8,https://www.canadiantire.ca/en/pdp/simoniz-3-l...,https://media-www.canadiantire.ca/product/auto...,10.0,car-care-accessories
815,Zinsser Bulls Eye 1-2-3 Water-Based Primer - S...,59.99,47.99,4.4,048-8344-2,https://www.canadiantire.ca/en/pdp/zinsser-bul...,https://media-www.canadiantire.ca/product/fixi...,20.0,paint
603,Braun BrewSense Programmable Coffee Maker w/ G...,149.99,119.99,4.2,043-2781-6,https://www.canadiantire.ca/en/pdp/braun-brews...,https://media-www.canadiantire.ca/product/livi...,20.0,kitchen


In [19]:
print(len(sale_df2))
sale_df2.sample(5)

461


,Product Name,Original Price,Sale Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
181,"Mothers Car Rubber, Vinyl & Plastic Protectant...",22.99,20.69,4.8,039-2536-4,https://www.canadiantire.ca/en/pdp/mothers-car...,https://media-www.canadiantire.ca/product/auto...,10.0,car-care-accessories
381,Pedigree Vitality+ Roasted Chicken and Vegetab...,25.99,23.39,4.2,042-6425-4,https://www.canadiantire.ca/en/pdp/pedigree-vi...,https://media-www.canadiantire.ca/product/livi...,10.0,pet-care
62,"FRANK Large Swift-Tie Recycling Bags, Blue Tra...",8.99,6.49,4.7,042-9825-4,https://www.canadiantire.ca/en/pdp/frank-large...,https://media-www.canadiantire.ca/product/livi...,28.0,cleaning
63,Mobil Delvac™ 1300 Super CK-4 15W40 Convention...,45.99,38.99,4.6,028-9937-4,https://www.canadiantire.ca/en/pdp/mobil-delva...,https://media-www.canadiantire.ca/product/auto...,15.0,auto-maintenance
304,Prestone Heavy-Duty 50/50 Premixed Anti-Freeze...,28.99,24.64,4.5,029-4118-2,https://www.canadiantire.ca/en/pdp/prestone-he...,https://media-www.canadiantire.ca/product/auto...,15.0,auto-maintenance


In [20]:
clearance_df.dtypes

Product Name         object
Original Price      float64
Clearance Price     float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [21]:
sale_df.dtypes

Product Name         object
Original Price      float64
Sale Price          float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [22]:
sale_df2.dtypes

Product Name         object
Original Price      float64
Sale Price          float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [23]:
clearance_df.drop('Image Link', axis=1, inplace=True)
sale_df.drop('Image Link', axis=1, inplace=True)
sale_df2.drop('Image Link', axis=1, inplace=True)

In [24]:
clearance_column_order = ['Product Name',
                    'Original Price',
                    'Clearance Price',
                    'Percentage Off',
                    'Product Category',
                    'Product Code',
                    'Rating',
                    'Link']

sale_column_order = ['Product Name',
                    'Original Price',
                    'Sale Price',
                    'Percentage Off',
                    'Product Category',
                    'Product Code',
                    'Rating',
                    'Link']

clearance_df = clearance_df.loc[:, clearance_column_order]
sale_df = sale_df.loc[:, sale_column_order]
sale_df2 = sale_df2.loc[:, sale_column_order]

In [31]:
clearance_df['Rating'] = clearance_df['Rating'].astype(float)
sale_df['Rating'] = sale_df['Rating'].astype(float)
sale_df2['Rating'] = sale_df2['Rating'].astype(float)

In [32]:
clearance_df.sample(5)

,Product Name,Original Price,Clearance Price,Percentage Off,Product Category,Product Code,Rating,Link
738,Stanley Dual Temp Full Size Glue Gun with Inte...,52.99,34.93,34.0,tools,054-0171-4,3.0,https://www.canadiantire.ca/en/pdp/stanley-dua...
849,Hoover Robot HEPA Replacement Filter,17.99,1.43,92.0,cleaning,043-8108-6,0.0,https://www.canadiantire.ca/en/pdp/hoover-robo...
504,Alpena 4 Pocket RV Bedside Shoe Organizer,17.99,7.93,56.0,automotive-outdoor-adventure,040-9741-6,4.0,https://www.canadiantire.ca/en/pdp/alpena-4-po...
150,Hamilton Beach Non-Stick Egg Bites Maker/Egg C...,49.99,26.93,46.0,kitchen,043-2771-0,4.6,https://www.canadiantire.ca/en/pdp/hamilton-be...
175,"Umbra Corsa Ceramic Soap Dispenser, White, 340-mL",10.99,5.93,46.0,home-decor,063-6263-8,4.8,https://www.canadiantire.ca/en/pdp/umbra-corsa...


In [33]:
sale_df.sample(5)

,Product Name,Original Price,Sale Price,Percentage Off,Product Category,Product Code,Rating,Link
171,Thermacell E5512-Hour DEET-Free Radius Zone Re...,69.99,59.99,14.0,gardening,159-1157-6,4.0,https://www.canadiantire.ca/en/pdp/thermacell-...
6,Outbound 3-Piece Hardside Spinner Wheel Travel...,459.99,159.99,65.0,camping,076-0860-2,3.5,https://www.canadiantire.ca/en/pdp/outbound-3-...
1690,MAXIMUM 10-in 60T Carbide Tipped Fine Finish C...,64.99,45.49,30.0,tools,054-2584-0,5.0,https://www.canadiantire.ca/en/pdp/maximum-10-...
258,Mastervac AT18116P 4.0 Peak HP Poly Wet/Dry Sh...,99.99,79.99,20.0,tools,054-0361-2,4.0,https://www.canadiantire.ca/en/pdp/mastervac-a...
783,simplicité Regular 4-Flap Ties Outdoor Garbage...,15.99,9.59,40.0,cleaning,042-8383-0,3.6,https://www.canadiantire.ca/en/pdp/simplicite-...


In [34]:
sale_df2.sample(5)

,Product Name,Original Price,Sale Price,Percentage Off,Product Category,Product Code,Rating,Link
352,Turtle Wax Hybrid Solutions Car Ceramic Acryli...,24.99,22.49,10.0,car-care-accessories,039-0551-8,4.4,https://www.canadiantire.ca/en/pdp/turtle-wax-...
301,Mothers California Gold Pure Brazilian Car Car...,25.99,23.39,10.0,,039-2863-4,4.7,https://www.canadiantire.ca/en/pdp/mothers-cal...
254,Stoner Invisible Glass Premium Car Glass Clean...,10.99,9.89,10.0,car-care-accessories,039-0432-2,4.7,https://www.canadiantire.ca/en/pdp/stoner-invi...
336,Turtle Wax Hybrid Solutions Hyper Foam Wheel C...,17.99,16.19,10.0,car-care-accessories,039-3665-0,4.3,https://www.canadiantire.ca/en/pdp/turtle-wax-...
189,"All-Purpose Push Broom with Metal Handle, 18-in",24.99,14.99,40.0,cleaning,042-9434-4,3.7,https://www.canadiantire.ca/en/pdp/all-purpose...


In [35]:
clearance_df.to_csv("./clearance.csv", index=False)
sale_df.to_csv("./sale.csv", index=False)
sale_df2.to_csv("./sale2.csv", index=False)

In [ ]:
# from datetime import datetime

# current_date = datetime.now().strftime("%Y-%m-%d")

# clearance_df.to_csv(f"./clearance_{current_date}.csv", index=True)
# sale_df.to_csv(f"./sale_{current_date}.csv", index=True)
# sale_df2.to_csv(f"./sale2_{current_date}.csv", index=True)

In [36]:
clearance_df.columns

Index(['Product Name', 'Original Price', 'Clearance Price', 'Percentage Off',
       'Product Category', 'Product Code', 'Rating', 'Link'],
      dtype='object')